In [87]:
import pandas as pd
from pandasql import sqldf 
import datetime # cambio de formato de fecha
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler
import numpy as np
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px

pysqldf = lambda q: sqldf(q, globals())

df = pd.read_csv("facebook_df.csv", sep=";")

In [88]:
df.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


Podemos observar que los nombres de las columnas no un tanto confusos y podemos sacar palabras clave para renombrarlas e determinar mucho mejor su uso. 
En específico, las columnas con nombre de lifetime podemos sustituir con el prefijo 'lf' o acompañarlo de la primera letra de la palabra compún; como 't' para 'total' y 'p' para post'; y una palabra clave. 
* Lifetime Post Total Reach - ltpt Reach
* Lifetime Post Total Impressions - ltpt Impressions
* Lifetime Engaged Users - lt Engaged Users
* Lifetime Post Consumers - ltp Consumers
* Lifetime Post Compumptions - ltp Consumptions
* Lifetime Post Impressions by people who have liked your Page - ltpi liked page 
* Lifetime Post reach by people who like your Page - ltp reach pel page
* Lifetime People who have liked your Page and engaged with your post - lfpe liked and engaged post 

In [89]:
# Cambiamos el nombre a las columnas
df = df.rename(columns = { 'Page total likes':'TotalLikes',
                            'Post Month':'Month',
                            'Post Weekday':'Weekday',
                            'Post Hour':'Hour',
                            'Lifetime Post Total Reach':'Reach', 
                            'Lifetime Post Total Impressions':'Impressions', 
                            'Lifetime Engaged Users':'EngagedUsers',
                            'Lifetime Post Consumers':'Consumers',
                            'Lifetime Post Consumptions': 'Consumptions',
                            'Lifetime Post Impressions by people who have liked your Page':'Followers',
                            'Lifetime Post reach by people who like your Page':'LikeFollowers',
                            'Lifetime People who have liked your Page and engaged with your post':'EngagedFollowers',
                            'Total Interactions':'TotalInteract'})
df.head()

,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TotalLikes        500 non-null    int64  
 1   Type              500 non-null    object 
 2   Category          500 non-null    int64  
 3   Month             500 non-null    int64  
 4   Weekday           500 non-null    int64  
 5   Hour              500 non-null    int64  
 6   Paid              499 non-null    float64
 7   Reach             500 non-null    int64  
 8   Impressions       500 non-null    int64  
 9   EngagedUsers      500 non-null    int64  
 10  Consumers         500 non-null    int64  
 11  Consumptions      500 non-null    int64  
 12  Followers         500 non-null    int64  
 13  LikeFollowers     500 non-null    int64  
 14  EngagedFollowers  500 non-null    int64  
 15  comment           500 non-null    int64  
 16  like              499 non-null    float64
 1

In [91]:
# Aquí obtenemos específicamente qúe renglones tienen valores faltantes, para identificar si son datos relevantes para el cliente. 
nan_rows = df[df.isnull().any(1)]
nan_rows

/tmp/ipykernel_27647/2546253675.py:2: FutureWarning:

In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.



,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
111,136736,Photo,1,10,6,8,0.0,1261,2158,37,37,49,1911,1077,33,0,NaN,NaN,0
120,136393,Photo,1,10,7,9,0.0,584,1029,273,271,308,943,511,232,0,2.0,NaN,2
124,136393,Photo,1,10,7,6,0.0,677,1285,251,246,297,1210,615,211,0,7.0,NaN,7
164,135428,Photo,1,9,5,10,0.0,1060,2004,266,251,337,1705,870,204,0,18.0,NaN,18
499,81370,Photo,2,1,4,4,NaN,4188,7292,564,524,743,3861,2200,316,0,91.0,28.0,119


In [92]:
# Remplazaremos los nans con '0' para poder utilizar la columna más versatilmente 
df[df['Paid'].isnull()]
df['Paid'] = df['Paid'].fillna(0)

df[df['like'].isnull()]
df['like'] = df['like'].fillna(0)
df['like'] = df['like'].astype(int)

df[df['share'].isnull()]
df['share'] = df['share'].fillna(0)
df['share'] = df['share'].astype(int)

# ¿Cambiar nombre de categorías?
# 1 == action
# 2 == product
# 3 == inspiration


In [93]:
# Revisamos que se haya cambiado correctamente
df.loc[111:'like'].head(1)

,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
111,136736,Photo,1,10,6,8,0.0,1261,2158,37,37,49,1911,1077,33,0,0,0,0


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TotalLikes        500 non-null    int64  
 1   Type              500 non-null    object 
 2   Category          500 non-null    int64  
 3   Month             500 non-null    int64  
 4   Weekday           500 non-null    int64  
 5   Hour              500 non-null    int64  
 6   Paid              500 non-null    float64
 7   Reach             500 non-null    int64  
 8   Impressions       500 non-null    int64  
 9   EngagedUsers      500 non-null    int64  
 10  Consumers         500 non-null    int64  
 11  Consumptions      500 non-null    int64  
 12  Followers         500 non-null    int64  
 13  LikeFollowers     500 non-null    int64  
 14  EngagedFollowers  500 non-null    int64  
 15  comment           500 non-null    int64  
 16  like              500 non-null    int64  
 1

In [95]:
df['TotalLikes'].max()

139441

In [96]:
df['TotalLikes'].min()

81370

Ya que vemos que no hay espacios en blanco podemos iniciar cierto análisis. 
Empezaremos eligiendo las variables de mayor importancia para el cliente.
* Que tiempos presentan mayor impacto (mes, día, hora)
    * Mes nos ayuda a identificar tendencias por TEMPORADA durante el año. 
    * Día nos ayuda a determinar comportamiento a lo largo de la semana. 
    * Hora sirve para identificar momentos de mayor tráfico en la página por día. 
* Que tipos de interacciones se encuentran (likes, compartir, comentarios, interacciones totales)
    * Likes son una muestra clara de respuesta/interés a un post. 
    * Comentarios demuestran interés mayor a solo observar el post, puede fomentar aún más interacción.
    * Compartir nos indica un interés del consumidor en extender el alcance del post. 
* Quienes interactuan con los posts (usuarios generales, seguidores de la página) y cómo. 
    * Usuarios generales pueden ayudar a identificar TENDENCIAS ATRACTIVAS para el público común. 
    * Seguidores de la página nos muestran QUÉ LOS MANTIENE INTERESADOS. 

In [97]:
# Una primera métrica para identificar tendencia: ¿Qué tipo de post contiene el mayor tipo de presencia? 
presdf = pysqldf("""
SELECT Type, TotalLikes, comment, share, TotalInteract
FROM df
GROUP BY Type 
""")
presdf

,Type,TotalLikes,comment,share,TotalInteract
0,Link,138414,0,2,18
1,Photo,139441,4,17,100
2,Status,139441,5,29,164
3,Video,138895,18,13,174


In [98]:
# 
tpl = sqldf("""
SELECT TotalLikes, COUNT (TotalLikes) count
FROM df
GROUP BY TotalLikes
ORDER BY count
""")
tpl

,TotalLikes,count
0,91009,1
1,91116,1
2,91223,1
3,91330,1
4,91437,1
...,...,...
85,138895,14
86,139441,14
87,129600,15
88,124940,17


In [99]:
# Conteo de likes mensualmente
monthdf = sqldf("""
SELECT Month, like, COUNT(like) as count
FROM df
GROUP BY Month 
ORDER BY Month
""")
monthdf

,Month,like,count
0,1,65,25
1,2,234,26
2,3,63,36
3,4,179,50
4,5,79,37
5,6,204,49
6,7,95,52
7,8,766,34
8,9,54,36
9,10,153,60


In [100]:
data0 = go.Scatter(
    x = monthdf.Month,
    y = monthdf.count(),
    name = 'Mes',
    text = monthdf.count(),
    mode = 'markers + lines'
)
data1 = go.Bar(
    x = monthdf.Month,
    y = monthdf.like,
    name = 'Mes',
    textposition = 'outside',
    texttemplate = '%(text:.2f)'
)

In [101]:
data = [data0, data1]

layout = go.Layout(title = 'Conteo de likes por mes', barmode = 'stack')

figure = go.Figure(data = data, layout = layout)

In [102]:
figure.show()

In [103]:
# Cambiamos el nombre de los días de la semana
df['Weekday'] = df['Weekday'].astype(str)
df['Weekday'] = df['Weekday'].replace('1', 'SUN')
df['Weekday'] = df['Weekday'].replace('2', 'MON')
df['Weekday'] = df['Weekday'].replace('3', 'TUE')
df['Weekday'] = df['Weekday'].replace('4', 'WED')
df['Weekday'] = df['Weekday'].replace('5', 'THU')
df['Weekday'] = df['Weekday'].replace('6', 'FRI')
df['Weekday'] = df['Weekday'].replace('7', 'SAT')

df

,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
0,139441,Photo,2,12,WED,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79,17,100
1,139441,Status,2,12,TUE,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130,29,164
2,139441,Photo,3,12,TUE,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66,14,80
3,139441,Photo,2,12,MON,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572,147,1777
4,139441,Photo,2,12,MON,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325,49,393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,85093,Photo,3,1,SAT,2,0.0,4684,7536,733,708,985,4750,2876,392,5,53,26,84
496,81370,Photo,2,1,THU,8,0.0,3480,6229,537,508,687,3961,2104,301,0,53,22,75
497,81370,Photo,1,1,THU,2,0.0,3778,7216,625,572,795,4742,2388,363,4,93,18,115
498,81370,Photo,3,1,WED,11,0.0,4156,7564,626,574,832,4534,2452,370,7,91,38,136


In [104]:
# Conteo de likes mensualmente
reachdf = sqldf("""
SELECT Weekday, like, COUNT(like) as count
FROM df
GROUP BY Weekday 
ORDER BY Weekday
""")
reachdf

,Weekday,like,count
0,FRI,113,81
1,MON,1572,66
2,SAT,325,82
3,SUN,152,68
4,THU,233,67
5,TUE,130,64
6,WED,79,72


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TotalLikes        500 non-null    int64  
 1   Type              500 non-null    object 
 2   Category          500 non-null    int64  
 3   Month             500 non-null    int64  
 4   Weekday           500 non-null    object 
 5   Hour              500 non-null    int64  
 6   Paid              500 non-null    float64
 7   Reach             500 non-null    int64  
 8   Impressions       500 non-null    int64  
 9   EngagedUsers      500 non-null    int64  
 10  Consumers         500 non-null    int64  
 11  Consumptions      500 non-null    int64  
 12  Followers         500 non-null    int64  
 13  LikeFollowers     500 non-null    int64  
 14  EngagedFollowers  500 non-null    int64  
 15  comment           500 non-null    int64  
 16  like              500 non-null    int64  
 1

In [106]:
# Cambiamos los meses
df['Month'] = df['Month'].astype(str)
df['Month'] = df['Month'].replace('1', 'JAN')
df['Month'] = df['Month'].replace('2', 'FEB')
df['Month'] = df['Month'].replace('3', 'MAR')
df['Month'] = df['Month'].replace('4', 'APR')
df['Month'] = df['Month'].replace('5', 'MAY')
df['Month'] = df['Month'].replace('6', 'JUN')
df['Month'] = df['Month'].replace('7', 'JUL')
df['Month'] = df['Month'].replace('8', 'AUG')
df['Month'] = df['Month'].replace('9', 'SEP')
df['Month'] = df['Month'].replace('10', 'OCT')
df['Month'] = df['Month'].replace('11', 'NOV')
df['Month'] = df['Month'].replace('12', 'DEC')
df

,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
0,139441,Photo,2,DEC,WED,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79,17,100
1,139441,Status,2,DEC,TUE,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130,29,164
2,139441,Photo,3,DEC,TUE,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66,14,80
3,139441,Photo,2,DEC,MON,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572,147,1777
4,139441,Photo,2,DEC,MON,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325,49,393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,85093,Photo,3,JAN,SAT,2,0.0,4684,7536,733,708,985,4750,2876,392,5,53,26,84
496,81370,Photo,2,JAN,THU,8,0.0,3480,6229,537,508,687,3961,2104,301,0,53,22,75
497,81370,Photo,1,JAN,THU,2,0.0,3778,7216,625,572,795,4742,2388,363,4,93,18,115
498,81370,Photo,3,JAN,WED,11,0.0,4156,7564,626,574,832,4534,2452,370,7,91,38,136


In [107]:
# Conteo de likes mensualmente
monthdf = sqldf("""
SELECT Month, like, COUNT(like) as count
FROM df
GROUP BY Month 
ORDER BY count
""")
monthdf

,Month,like,count
0,JAN,65,25
1,FEB,234,26
2,AUG,766,34
3,MAR,63,36
4,SEP,54,36
5,MAY,79,37
6,NOV,48,45
7,JUN,204,49
8,APR,179,50
9,DEC,79,50


In [108]:
# Categories
df['Category'] = df['Category'].astype(str)
df['Category'] = df['Category'].replace('1', 'action')
df['Category'] = df['Category'].replace('2', 'product')
df['Category'] = df['Category'].replace('3', 'inspiration')

df

,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
0,139441,Photo,product,DEC,WED,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79,17,100
1,139441,Status,product,DEC,TUE,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130,29,164
2,139441,Photo,inspiration,DEC,TUE,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66,14,80
3,139441,Photo,product,DEC,MON,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572,147,1777
4,139441,Photo,product,DEC,MON,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325,49,393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,85093,Photo,inspiration,JAN,SAT,2,0.0,4684,7536,733,708,985,4750,2876,392,5,53,26,84
496,81370,Photo,product,JAN,THU,8,0.0,3480,6229,537,508,687,3961,2104,301,0,53,22,75
497,81370,Photo,action,JAN,THU,2,0.0,3778,7216,625,572,795,4742,2388,363,4,93,18,115
498,81370,Photo,inspiration,JAN,WED,11,0.0,4156,7564,626,574,832,4534,2452,370,7,91,38,136


In [109]:
#Paid
df['Paid'] = df['Paid'].astype(str)
df['Paid'] = df['Paid'].replace('0.0', 'No')
df['Paid'] = df['Paid'].replace('1.0', 'Si')





In [110]:
categorydf = pysqldf("""
SELECT Category, COUNT(like) as count
FROM df
GROUP BY Category
ORDER BY COUNT
""")

categorydf

,Category,count
0,product,130
1,inspiration,155
2,action,215


In [111]:
fig = px.pie(categorydf, names='Category', values='count')
fig.show()

# Mi Código

In [112]:
df.head(20)

,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
0,139441,Photo,product,DEC,WED,3,No,2752,5091,178,109,159,3078,1640,119,4,79,17,100
1,139441,Status,product,DEC,TUE,10,No,10460,19057,1457,1361,1674,11710,6112,1108,5,130,29,164
2,139441,Photo,inspiration,DEC,TUE,3,No,2413,4373,177,113,154,2812,1503,132,0,66,14,80
3,139441,Photo,product,DEC,MON,10,Si,50128,87991,2211,790,1119,61027,32048,1386,58,1572,147,1777
4,139441,Photo,product,DEC,MON,3,No,7244,13594,671,410,580,6228,3200,396,19,325,49,393
5,139441,Status,product,DEC,SUN,9,No,10472,20849,1191,1073,1389,16034,7852,1016,1,152,33,186
6,139441,Photo,inspiration,DEC,SUN,3,Si,11692,19479,481,265,364,15432,9328,379,3,249,27,279
7,139441,Photo,inspiration,DEC,SAT,9,Si,13720,24137,537,232,305,19728,11056,422,0,325,14,339
8,139441,Status,product,DEC,SAT,3,No,11844,22538,1530,1407,1692,15220,7912,1250,0,161,31,192
9,139441,Photo,inspiration,DEC,FRI,10,No,4694,8668,280,183,250,4309,2324,199,3,113,26,142


In [113]:
"""
Comparación de porcentaje de reach con engaged users

Qué porcentaje de diff existe entre followers que dieron like y nuevos usuarios.(usar engagedusers y engagedfollowers)


Qué tipo de categoría es cuando la gente tiene más interacciones?

sirven más impresiones para que den like?


"""

'\nComparación de porcentaje de reach con engaged users\n\nQué porcentaje de diff existe entre followers que dieron like y nuevos usuarios.(usar engagedusers y engagedfollowers)\n\n\nQué tipo de categoría es cuando la gente tiene más interacciones?\n\nsirven más impresiones para que den like?\n\n\n'

In [114]:
#En esta tabla se muestra la diferencia de alcance de pagar o no en la plataforma de facebook
df1 = df.groupby(['Paid']).mean()
df1=df1.reset_index()
df1[['Paid','Reach','EngagedUsers']]

/tmp/ipykernel_27647/1322056240.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Paid,Reach,EngagedUsers
0,No,11829.988920,848.65651
1,Si,19288.158273,1106.52518


In [115]:
fig = px.pie(df1, names='Paid', values='EngagedUsers', hole=0.5)
fig.show()

In [116]:
data0 = go.Bar(
    x = df1.EngagedUsers,
    y = df1.Paid,
    name = 'EngagedUsers',
    textposition = 'outside',
    texttemplate = '%(text:.2f)'
)
data1 = go.Bar(
    x = df1.Reach,
    y = df1.Paid,
    name = 'Reach',
    textposition = 'outside',
    texttemplate = '%(text:.2f)'
)

In [117]:
data = [data0, data1]

layout = go.Layout(title = 'Publicidad pagada')

figure = go.Figure(data = data, layout = layout)
figure.show()

In [118]:
#Aquí se muestran el porcentaje de los usuarios que han dado click del total de alcance

df2 = df.groupby(['Type', 'Category']).mean()
df2=df2.reset_index()
df2['EngagedUsers%']= (df2['EngagedUsers']/df2['Reach'])*100
df2.sort_values(by=['EngagedUsers%'], inplace = True, ascending = False)
df2[['Type','Category','EngagedUsers%']]

/tmp/ipykernel_27647/4030079611.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Type,Category,EngagedUsers%
8,Status,product,16.141701
6,Status,action,12.566629
7,Status,inspiration,11.812367
5,Photo,product,8.587449
4,Photo,inspiration,7.396529
3,Photo,action,5.028020
1,Link,inspiration,4.101222
9,Video,action,3.333612
0,Link,action,1.810180
2,Link,product,1.797557


In [119]:
#En esta tabla se muestra que la mayorías de los clicks son realizados por los followers y mayores que usuarios 
#nuevos usuarios que han dado click. 
df2['EngagedNew']= df2['EngagedUsers']-df2['EngagedFollowers']
df2['EngagedNew%'] = (df2['EngagedNew']/df2['EngagedUsers'])*100
df2['EngagedFollowers%'] = (df2['EngagedFollowers']/df2['EngagedUsers'])*100
df2.sort_values(by=['EngagedFollowers'], inplace = True, ascending = False)
df3 = df2.groupby(['Type', 'Category']).sum()
df3=df3.reset_index()
#Creo una nueva columna para saber cuántos usuarios que no han dado like a la página dan click a cualquier cosa del 
#post
df3[['Type','Category','EngagedNew%','EngagedFollowers%']]

,Type,Category,EngagedNew%,EngagedFollowers%
0,Link,action,40.184435,59.815565
1,Link,inspiration,8.156028,91.843972
2,Link,product,10.679612,89.320388
3,Photo,action,38.689094,61.310906
4,Photo,inspiration,36.971040,63.028960
5,Photo,product,38.586497,61.413503
6,Status,action,11.375321,88.624679
7,Status,inspiration,15.549252,84.450748
8,Status,product,16.040045,83.959955
9,Video,action,42.622814,57.377186


In [120]:
#Esta tabla nos ayuda a identificar qué tipo de categoría de promocionar un producto, genera más interacción del
#usuario en el post
df4 = df2.groupby(['Type']).sum()
df4=df4.reset_index()
df4.sort_values(by=['EngagedUsers'], inplace = True, ascending = False)
df4[['Type','EngagedUsers']]

/tmp/ipykernel_27647/4214161155.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Type,EngagedUsers
2,Status,5194.377193
1,Photo,2412.989152
3,Video,1707.000000
0,Link,742.850000


In [121]:
fig = px.pie(df4, names='Type', values='EngagedUsers')
fig.show()

In [122]:
#Mientras las impresiones son mayores, el porcentaje de clicks es mayor?
df2.sort_values(by=['Impressions'], inplace = True, ascending = False)
df2[['Type','Category','Impressions','EngagedUsers%']]

,Type,Category,Impressions,EngagedUsers%
9,Video,action,102622.428571,3.333612
3,Photo,action,39286.308108,5.028020
0,Link,action,30549.600000,1.810180
6,Status,action,29228.000000,12.566629
4,Photo,inspiration,25016.753333,7.396529
8,Status,product,24754.947368,16.141701
7,Status,inspiration,15657.250000,11.812367
5,Photo,product,14628.307692,8.587449
1,Link,inspiration,10885.000000,4.101222
2,Link,product,10083.000000,1.797557
